In [1]:
# import metrics
import os
import math
import random
import time
from PIL import Image

import torch # pytorch의 tensor와 그와 관련된 기본 연산 등을 지원
import torch.nn as nn # 여러 딥러닝 layer와 loss, 함수 등을 클래스 형태로 지원
import torch.nn.functional as F # 여러 loss, 함수 등을 function 형태로 지원
import torch.optim as optim # 여러 optimizer를 지원
import torchvision.models as models

dev = 'cuda' if torch.cuda.is_available() else 'cpu'

### CosFace

![Loss function](https://drive.google.com/uc?id=15i-zpo60EgKsgvqsSljzaGhr9vzqrmd2)

ArcFace와 비슷한 모델 중 하나로 CosFace라는 모델이 있습니다. 위 Loss function은 CosFace의 loss function에 해당합니다. 위 loss function을 참고하여 아래의 ??? 부분을 채워주세요.

In [4]:
'''
########################## 5개 ##############################
'''
class CosMarginProduct(nn.Module):
    '''
    목적 : Cosface 의 last fc layer의 구현
    
    인자 :
    in_features : feature의 dimension
    out_features : class 개수
    '''
    def __init__(self, in_features, out_features, s=30.0, m=0.1):
        super(CosMarginProduct, self).__init__()        
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        
        # fc의 parameter 만들기 : (in_features x out_features)의 크기를 갖는 FloatTensor로 만들 것
        self.weight = torch.nn.Parameter(torch.FloatTensor(in_features, out_features))
        
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        '''
        Step 1. cos(theta)-m 계산하기
        '''

        # cos_theta = (x / ||x||) * (w * ||w||) 를 이용해 cosine_theta 구하기
        # 어느 dimension을 기준으로 normalization을 해서 torch.mm에 넘겨줘야 할까요?
        cos = torch.mm(F.normalize(input, dim=1), F.normalize(self.weight, dim=0))
        
        # sin = torch.sqrt((1 - torch.pow(cosine, 2)).clamep(0, 1))
        
        # cos_theta - m 구하기
        cos_m = cos - self.m
        
        '''
        Step 2. cos(theta)-m 에서 dim=1에 기준으로 y_i에 해당하는 부분만 남기고 나머지는 cos(theta)로 되돌리기 
        '''
        one_hot = torch.zeros(cos.size()).to(dev)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * cos_m) + ((1 - one_hot) * cos)
        '''
        Step 3. 최종 output 계산하기
        '''
        output *= self.s
        
        return output

In [5]:
debug = CosMarginProduct(500, 5, s=30.0, m=0.1)
inputs = torch.randn(6, 500)
labels = torch.ones(6)*1

debug(inputs, labels)

tensor([[-0.5679, -3.3301,  2.4454,  1.3051, -0.3312],
        [ 1.4849, -2.4852, -0.1755, -0.2375, -0.7200],
        [ 0.4409, -4.2070,  0.9451, -0.9159,  0.5482],
        [ 1.3983, -2.0652, -1.3277, -1.3929,  0.2917],
        [ 1.3694, -1.8475,  2.9558,  3.2490,  0.2565],
        [-1.8637, -2.0020, -1.2830, -0.7428, -2.4385]], grad_fn=<MulBackward0>)

### SphereFace

![Loss function](https://drive.google.com/uc?id=1qWYN2ATdCfy8ct-Wru8lWKdy2pof-QO9)

ArcFace와 비슷한 모델 중 하나로 SphereFace 모델이 있습니다. 위 Loss function은 SphereFace의 loss function에 해당합니다. 위 loss function을 참고하여 아래의 ??? 부분을 채워주세요.

In [6]:
'''
########################## 5개 ##############################
'''
class SphereMarginProduct(nn.Module):
    '''
    목적 : Sphereface의 last fc layer의 구현
    
    인자 :
    in_features : feature의 dimension
    out_features : class 개수
    '''
    def __init__(self, in_features, out_features, m=4):
        super(SphereMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        
        # fc의 parameter 만들기 : (out_features x in_features)의 크기를 갖는 FloatTensor로 만들 것
        self.weight = torch.nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        '''
        Step 1. cos(m * theta) 계산하기
        '''

        # cos_theta = (x / ||x||) * (w * ||w||) 를 이용해 cosine_theta 구하기
        # 어느 dimension을 기준으로 normalization을 해서 F.linear에 넘겨줘야 할까요?
        cos = F.linear(F.normalize(input, dim=1), F.normalize(self.weight, dim=1))
        # cos(m * theta) 구하기. 논문에서 m=4로 제시하고 있으므로 m=4 일 경우에 대해서만 계산합니다.
        # 효율성을 위해 arccos 등의 다른 연산 없이 위에서 얻은 cos만을 사용해 계산합니다.
        # cos (2 * theta) = 2 * cos(\theta) * cos(\theta) - 1
        # cos (4 * theta) = 2 * cos(2 * theta) * cos(2 * theta) = ~~~
        cos_m = 1 - 8*(cos**2) + 8*(cos**4)
        
#         --> cos2 = 2*cos*cos - 1
#         --> cos_m = 2*cos2*cos2 - 1
        
        '''
        Step 2. cos(m * theta) 에서 dim=1에 기준으로 y_i에 해당하는 부분만 남기고 나머지는 cos(theta)로 되돌리기 
        '''
#         one_hot = torch.zeros(cos.size()).to('cpu')
#         one_hot.scatter_(1, label.view(-1, 1).long(), 1)
#         output = (????? * cos_m) + (????? * cos)
        one_hot = F.one_hot(label.long(), num_classes=self.out_features)
        output = (one_hot * cos_m) + ((1.0 - one_hot) * cos)
        '''
        Step 3. 최종 output 계산하기
        '''
        '''
        ##### 둘 다 맞아야 1개로 인정 #####
        '''
        x_norm = torch.norm(input, p='fro', dim=1) # fro: Frobenius norm

        output *= x_norm.unsqueeze(dim=1)
        return output


In [7]:
import torch
import numpy as np
input = torch.ones(20, 10)
x_norm = np.linalg.norm(input,ord=2)
print(x_norm)

14.142136


In [8]:
x = torch.randn(20, 10)
x_norm = torch.norm(x, dim=1).unsqueeze(dim=1)
x *= x_norm

### Backbone network

ResNet-101을 이용하여 Backbone network를 구현합니다. 아래 코드의 ??? 부분을 채워주세요.

In [21]:
'''
########################## 4개 ##############################
'''
class FeatureNet_101(nn.Module):
    def __init__(self, dim_feature):
        super(FeatureNet_101, self).__init__()
        resnet = models.resnet101(pretrained=False)
        
        # resnet.conv1 = nn.Conv2d(1, 64, (7,7), padding=(3,3))

        # resnet-101의 마지막 Conv layer block 까지 남기고 그 뒷부분은 잘라냅니다.
        # resnet-101 의 구조를 print해서 어디를 잘라야할 지 알 수 있습니다.
        self.backbone = nn.Sequential(* list(resnet.children())[0:-2])
        
        # 마지막 Conv layer block 부분 이후로 붙는 layer들입니다.
        # resnet-101 의 구조를 print해서 어떻게 뒤 쪽 layer를 디자인 해야할 지 알 수 있습니다.
        self.bn_4 = nn.BatchNorm2d(2048)
        self.dropout = nn.Dropout()
        self.fc = nn.Linear(2048 * 4 * 4, dim_feature)
        self.bn_5 = nn.BatchNorm1d(dim_feature)
        
    def forward(self, x):
        out = self.backbone(x)
        # print(out.shape) # layer 크기 구하기 
        out = self.bn_4(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)

        out = self.fc(out)
        out = self.bn_5(out)
        return out
    
# debug_resnet = FeatureNet_101(512)
# imgs = torch.randn(3, 3, 64, 64)
# rr = debug_resnet(imgs)

In [11]:
2048*4*4

32768

### FaceNet

위에서 구현한 각 모델의 마지막 FC layer들과 Backbone network를 합쳐서 하나의 얼굴인식모델을 만듭니다.
아래 코드의 ??? 부분을 채워주세요.

In [22]:
'''
########################## 1개 ##############################
'''
class FaceNet(nn.Module):
    '''
    ArcMarginProduct와 FeatureNet-50 을 결합한 ArcFace 모델의 구현
    '''
    def __init__(self, feature_dim, cls_num, model_type='Cosface'):
        super(FaceNet, self).__init__()
        self.feature_net = FeatureNet_101(feature_dim)
        
        if model_type == 'Cosface':
            self.classifier = CosMarginProduct(feature_dim, cls_num)
        elif model_type == 'Sphereface':
            self.classifier = SphereMarginProduct(feature_dim, cls_num)

    # 끝까지 Forward 하여 logit을 return
    '''
    ##### 둘 다 맞아야 1개로 인정 #####
    '''
    def forward(self, x, label):
        out = classifier(self.feature_net(x), label)
        return out
    
    # Feature를 extract
    def extract_feature(self, x):
        out = self.feature_net(x)
        return out

In [23]:
# 두 input 이미지의 유사도를 측정하는데 사용되는 cosine similarity

def cos_dist(x1, x2):
    return torch.sum(x1 * x2) / (torch.norm(x1) * torch.norm(x2))

### FaceNet

FaceNet을 이용하여 두 input 사이의 similarity를 계산합니다.

In [24]:
'''
########################## 1개 ##############################
'''
'''
##### 전체 다 맞아야 하나로 인정 #####
'''

# 두 input입니다.
x_1 = torch.randn(1, 3, 128, 128).to(dev)
x_2 = torch.randn(1, 3, 128, 128).to(dev)

# 각 model을 만듭니다. 이 모델에서 사용하는 feature의 dim은 512고 class는 총 1000개가 있습니다.
SphereFaceNet = FaceNet(feature_dim = 512, cls_num = 1000, model_type = 'Sphereface').to(dev)
CosFaceNet = FaceNet(feature_dim = 512, cls_num = 1000, model_type = 'Cosface').to(dev)


# test를 위해 model을 test phase로 변경합니다.
# 특정 layer는 training과 test 떄 다르게 동작하므로 이 설정은 필수입니다. (ex. dropout, batchnorm ...) 

# net.train()
# net(tensor(1,3,256,256)) ==> 안돌아갈 시,
# net.eval()
# net(tensor(1,3,256,256))

SphereFaceNet.eval()
CosFaceNet.eval()


# x_1, x_2로부터 SphereFace 모델을 이용해 feature를 추출합니다.
feature_1 = SphereFaceNet.extract_feature(x_1)
feature_2 = SphereFaceNet.extract_feature(x_2)

# 두 feature의 유사도를 계산합니다.
sim = cos_dist(feature_1, feature_2)
print('SphereFace에서 두 input의 유사도는 %f 입니다.' % sim.item())

# x_1, x_2로부터 CosFace 모델을 이용해 feature를 추출합니다.
feature_1 = CosFaceNet.extract_feature(x_1)
feature_2 = CosFaceNet.extract_feature(x_2)

# 두 feature의 유사도를 계산합니다.
sim = cos_dist(feature_1, feature_2)
print(f'CosFace에서 두 input의 유사도는 {sim.item()} 입니다.')

SphereFace에서 두 input의 유사도는 0.999710 입니다.
CosFace에서 두 input의 유사도는 0.999352216720581 입니다.
